# SERIMATS Application


|           |                |
| : ----- : | :------------: |
| Candidate | Matthieu David |
| stream    | developmental interpretability |
| subject   | [task variability](https://devinterp.com/projects/task-variability) |



This notebook is inspired from the notebook [RLCT Estimation of MNIST](https://colab.research.google.com/github/timaeus-research/devinterp/blob/main/examples/mnist.ipynb) with large change to fit my submission.


---
## RLCT Estimation of MNIST

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()

This Jupyter Notebook aims to reproduce the results of Lau et al. (2023) by measuring the Real Log Canonical Threshold (RLCT) for a small 2-layer ReLU model (about 1M parameters) trained on the MNIST dataset. It uses both Stochastic Gradient Nose-Hoover Thermostat (SGNHT) and Stochastic Gradient Langevin Dynamics (SGLD) as sampling methods.

---

## Table of Contents:
* packages installation
* Import for the all notebook
* Method definitions
* Experiments 1 (XP1): Impact of the number of classes
    * Toy model and Resnet18 instances
    * Settings and runs label sets $Card(\mathcal{L}abels) = 2$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 3$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 4$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 5$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 6$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 10$
* Displays $λ\ vs\ \mathcal{L}abels$
* Experiments 2 (XP2): Impact of the data diversity
* Experiments 3 (XP3): Impact of the number of classes

---

## Package installation:

In [ ]:
%pip install devinterp matplotlib seaborn torchvision

## Import for all the notebook

In [ ]:
import copy
import matplotlib.pyplot as plt

import os
from pydantic.types import confloat, conint
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from typing import Literal

from torchvision import datasets, transforms
from torchvision.models import resnet18

from devinterp.optim.sgld import SGLD
from devinterp.optim.sgnht import SGNHT

PRIMARY, SECONDARY, TERTIARY = sns.color_palette("muted")[:3]
PRIMARY_LIGHT, SECONDARY_LIGHT, TERTIARY_LIGHT = sns.color_palette("pastel")[:3]
sns.set_theme("paper")

## Method definitions

#### Class of the Toy model:
The toy model corresponds to the initial model used in the notebook RLCT Estimation of MNIST (rewrite a little for lisibility). This model is made of 2 linear layers, each activations being a ReLU function:

In [ ]:

# Define the toy neural network
class OriginalToyNet(nn.Module):
    def __init__(
        self,
        hidden_layer_sizes=[1024, 1024],
        input_dim=28 * 28,
        output_dim=10,
        activation=F.relu,
        with_bias=True,
    ):
        super(OriginalToyNet, self).__init__()
        self.input_dim = input_dim
        self.layer_sizes = [input_dim] + hidden_layer_sizes + [output_dim]
        self.activation = activation
        self.with_bias = with_bias
        self.layers = nn.ModuleList()
        for i in range(len(self.layer_sizes) - 1):
            dim_in, dim_out = self.layer_sizes[i : i + 2]
            self.layers.append(nn.Linear(dim_in, dim_out, bias=self.with_bias).float())

    def forward(self, x):
        x = x.view(-1, self.input_dim)
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        x = self.layers[-1](x)
        return x

# Define the toy neural network
class ToyNet(nn.Module):
    def __init__(
        self,
        hidden_layer_sizes=[1024, 1024],
        input_dim=28 * 28,
        output_dim=10,
        with_bias=True,
    ):
        super(ToyNet, self).__init__()
        self.input_dim = input_dim
        self.with_bias = with_bias
        self.layer_sizes = [input_dim] + hidden_layer_sizes + [output_dim]

        # First basic block = Linear + ReLU activation
        self.fc_1 = nn.Linear(in_features=input_dim, out_features=1024, bias=self.with_bias)
        self.relu_1 = nn.ReLU()

        # 2nd basic block = Linear + ReLU activation
        self.fc_2 = nn.Linear(in_features=1024, out_features=1024, bias=self.with_bias)
        self.relu_2 = nn.ReLU()

        # Last Fully Connected layer:
        self.fc_3 = nn.Linear(in_features=1024, out_features=output_dim, bias=self.with_bias)



    def forward(self, x):
        x = x.view(-1, self.input_dim)

        x = self.fc_1(x)
        x = self.relu_1(x)
        x = self.fc_2(x)
        x = self.relu_2(x)
        x = self.relu_2(x)

        x = self.fc_3(x)
        return x

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion):
    "Original train_one_epoch method from MNIST notebook of timaeus/devinterp"
    model.train()
    train_loss = 0
    for data, target in tqdm(train_loader):
        optimizer.zero_grad()
        output = model(data.to(DEVICE))
        loss = criterion(output, target.to(DEVICE))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    return train_loss / len(train_loader)


def evaluate(model, test_loader, criterion):
    "Original evaluate method from MNIST notebook of timaeus/devinterp"
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.to(DEVICE))
            loss = criterion(output, target.to(DEVICE))
            test_loss += loss.item()
    return test_loss / len(test_loader)


def train_step(model, optimizer, criterion, train_set):
    model.train()
    # model.to(DEVICE)

    train_loss, train_acc = 0.0, 0.0
    i = 0
    prog_bar = tqdm(train_set, desc=f"\ttrain_loss: {train_loss:0.2f} -- train_acc: {train_acc:0.2f} %")
    for x, y in prog_bar:
        # Stating variables
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        optimizer.zero_grad() # Clear gradients
        output = model(x) # Forward step
        loss = criterion(output, y) # Loss step

        # Backward step
        loss.backward()
        optimizer.step()

        # Accuracy
        top_p, prediction = output.topk(1, dim=1)
        equals = (prediction == y.view(*prediction.shape))

        prog_bar.set_description(f"\ttrain_loss: {train_loss / len(train_set) :0.5f} -- train_acc: {100 * train_acc / len(train_set) :0.2f} %")
        # Saving data
        train_loss += loss.item()
        train_acc += torch.mean(equals.type(torch.FloatTensor))

    return train_loss / len(train_set), train_acc / len(train_set)


def val_step(model, criterion, val_set):
    val_loss, val_acc = 0.0, 0.0

    model.eval()
    # model.to(DEVICE)
    with torch.no_grad():
        for i, (x, y) in enumerate(val_set):
            # Stating variables
            images = x.to(DEVICE)
            labels = y.to(DEVICE)

            output = model(images) # Forward step

            loss = criterion(output, labels) # Loss step

            # Accuracy
            top_p, prediction = output.topk(1, dim=1)
            equals = (prediction == labels.view(*prediction.shape))

            # Saving data
            val_loss += loss.item()
            val_acc += torch.mean(equals.type(torch.FloatTensor))

    return val_loss / len(val_set), val_acc / len(val_set)

def print_step_report(epoch:int, train_loss: float, val_loss: float, train_acc: float, val_acc: float):
    print(f"Epoch {epoch+1}:\n" \
          f"Train Loss: {train_loss:.5f}" \
          f"-- Val Loss: {val_loss:.5f}" \
          f"-- Train Acc: {train_acc:0.2f}" \
          f"-- Val Acc: {val_acc:0.2f}")
    
def plot_lambda_vs_epochs(train_losses, test_losses, rlct_estimates_sgnht, rlct_estimates_sgld):
    sns.set_style("whitegrid")
    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss", color=PRIMARY)
    ax1.plot(train_losses, label="Train Loss", color=PRIMARY)
    ax1.plot(test_losses, label="Test Loss", color=PRIMARY_LIGHT)
    ax1.tick_params(axis="y", labelcolor=PRIMARY)
    ax1.legend(loc="lower left")

    ax2 = ax1.twinx()
    ax2.set_ylabel(r"Local Learning Coefficient, $\hat \lambda$", color=SECONDARY)
    ax2.plot(rlct_estimates_sgnht, label="SGNHT", color=SECONDARY)
    ax2.plot(rlct_estimates_sgld, label="SGLD", color=SECONDARY_LIGHT)
    ax2.tick_params(axis="y", labelcolor=SECONDARY)
    ax2.legend(loc="center right")

    fig.tight_layout()
    plt.show()

## Experiments 1 (XP1): Impact of the number of classes

This section contains all the experiments about the XP1 detailled in the application report [here](link)

First basic code cells are given the dataset MNIST and basics training with the Toy model and Resnet18 are reported. The readers can explore the core process without modifying the code for the actual experiment.

Secondly each sub experiments possess a dedicated sub section , new data loaders are generated, where the labels are rewritten to performed the experiment with the dedicated label set cardinality.

### MNIST dataset

In [ ]:
# Load MNIST data
## definition of the transformation to apply to dataset
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.244, 0.225]),
    ]
)

## Load train data
batch_size = 512
train_data = datasets.MNIST("../data", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# Load test data
test_data = datasets.MNIST("../data", train=False, transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

### Toy model and Resnet18 instances

In [ ]:
# Depending of the run type, cuda or cpu mode is defined
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------------------------
N_CLASSES = 10 # for the 10 class digits of MNIST
lr = 0.005
n_epochs = 20
# --------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
model_resnet18 = resnet18(weights=None).to(DEVICE)

## changing the last fully connected layer of N neurons, each corresponding to a class.
new_fc_act = nn.Linear(model_resnet18.fc.in_features, N_CLASSES)
model_resnet18.fc = new_fc_act

## Changing the first conv2d layer to match the input image shape
model_resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Loss and optimizer definition
criterion_resnet18 = nn.CrossEntropyLoss()
optimizer_resnet18 = optim.SGD(model_resnet18.parameters(), lr=lr, momentum=0.9, nesterov=True)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# Initialize model, loss, optimizer and sgld sampler
model_toy = ToyNet().to(DEVICE)
criterion_toy = nn.CrossEntropyLoss()
optimizer_toy = optim.SGD(model_toy.parameters(), lr=lr, momentum=0.9, nesterov=True)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_toy = []
val_losses_toy = []
models_toy = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy.append(train_loss)
    val_losses_toy.append(val_loss)
    models_toy += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy, "toy_models.pth")

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet = []
val_losses_resnet = []
models_resnet = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet.append(train_loss)
    val_losses_resnet.append(val_loss)
    models_resnet += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet, "resnet_models.pth")

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
from devinterp.slt import estimate_learning_coeff

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

In [ ]:
plot_lambda_vs_epochs(train_losses_toy, val_losses_toy, rlct_estimates_sgnht, rlct_estimates_sgld)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ______________________    Resnet18 models    ______________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
from devinterp.slt import estimate_learning_coeff

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_resnet:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_resnet18,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_resnet18,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

### Settings and runs label sets $Card(\mathcal{L}abels)=2$

The different sets of labels to explore are: $\{1, other\}$ ; $\{2, other\}$ ; $\{4, other\}$ ; $\{5, other\}$ ; $\{9, other\}$.
`other` cannot be put as a label as it, it needs to be encoded into a integer. In order to relabel as we wish, we simply map the first label/digit of the sets to `1` and all the other to `0`. For example for the set $\{4,other\}$ it means that the images representing `4` will be relabeled as `1` and the images representing any of $[0, 1, 2, 3, 5, 6, 7, 8, 9]$ will be relabeled as `0`.

* labels modification and data loader
* Run for the toy model
* Run for the resnet18 model

In [ ]:
# preparing the output directories of XP1
dirs = ["XP1",
        "XP1/card_2-case_1", "XP1/card_2-case_2","XP1/card_2-case_3", "XP1/card_2-case_4", "XP1/card_2-case_5",
        "XP1/card_3-case_1", "XP1/card_3-case_2","XP1/card_3-case_3", "XP1/card_3-case_4", "XP1/card_3-case_5",
        "XP1/card_4-case_1", "XP1/card_4-case_2","XP1/card_4-case_3", "XP1/card_4-case_4", "XP1/card_4-case_5",
        "XP1/card_5-case_1", "XP1/card_5-case_2","XP1/card_5-case_3", "XP1/card_5-case_4", "XP1/card_5-case_5",
        "XP1/card_10-case_1", "XP1/card_10-case_2","XP1/card_10-case_3", "XP1/card_10-case_4", "XP1/card_10-case_5"]

for path in dirs:
        os.mkdir(path)

In [ ]:
def load_and_process_data(bs: int, mapping_label_to_encode: dict) -> tuple[DataLoader, DataLoader]:
    """ Loads MNIST dataset, relabel the images of train and test sets

    Function load MNIST dataset and performs the relabelling. The loading performs the transformation
    necessary to normalize the data distribution.
    Before forging the data loaders, the operation of relabelling to execute the experiments
    is performed.
    Arguments:
        bs: batch size, excepting a positive integer
        mapping_label_to_encode: mapping between original labels and target labels

    Returns:
        train_loader, test_loader: data loader of train and test
    """
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))])
    # Load train data
    train_data = datasets.MNIST("../data", train=True, transform=transform, download=True)
    for old_label, new_label in mapping_label_to_encode.items():
        train_data.targets[train_data.targets == old_label] = new_label
    print("set of labels in train data: ", train_data.targets.unique())
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Load test data
    test_data = datasets.MNIST("../data", train=False, transform=transform)
    for old_label, new_label in mapping_label_to_encode.items():
        test_data.targets[test_data.targets == old_label] = new_label
    print("set of labels in test data: ", test_data.targets.unique())
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

def models_factory(model_type: Literal["toy_model", "resnet18"],
                   optim_cfg: dict,
                   optim_name: Literal['SGD'] = "SGD",
                   criterion_name: Literal["CrossEntropyLoss"]="CrossEntropyLoss",
                   lr: confloat(gt=0, lt=1)=0.005,
                   n_epochs: conint(gt=0)=20,
                   nb_classes: conint(ge=2, le=10)=10
                   ):
    """Very light factory to handle the model instanciation.

    The main purpose of this method is to centralize the models instance
    allowing lighter code cells in the following.

    Note:
        versatility is very limited here, but the idea is to use a common interface to
        generate a model withing a collection of possibility

    Args:
        model_type: name of the model to return
        optim_name: optimiser name of the model
        optim_cfg: configuration of the optimiser
        criterion_name: Loss function associated with the model for training.
            Defaults to "CrossEntropyLoss".
        lr: learning rate. Defaults to 0.005.
        n_epochs: number of epochs. Defaults to 20.
        nb_classes: number of classes of the task.
    """
    if model_type == "toy_model":
        model_toy = ToyNet(output_dim=2).to(DEVICE)
    elif model_type == "resnet18":
        model_resnet18 = resnet18(weights=None).to(DEVICE)

        ## changing the last fully connected layer of N neurons, each corresponding to a class.
        new_fc_act = nn.Linear(model_resnet18.fc.in_features, N_CLASSES)
        model_resnet18.fc = new_fc_act

        ## Changing the first conv2d layer to match the input image shape
        model_resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    else:
        raise ValueError(f"Model factory does not handle model_type: '{model_type}'")
        
    # Loss definition
    if criterion_name == "CrossEntropyLoss":
        criterion = nn.CrossEntropyLoss()
    else:
        ValueError("Model factory only handles Cross Entropy Loss.")
    
    # Optimizer definition
    if optim_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, **optim_cfg)
    else:
        ValueError("Model factory only handles Cross Entropy Loss.")

    return model, optimizer, criterion

In [ ]:
# keys are the original label, values are the new labels
map_label_card_2_1 = {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_2 = {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_3 = {1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_4 = {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_5 = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1}

##### 1.1 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{1, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_1)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}

model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# train_losses_resnet_xp1_1 = []
# val_losses_resnet_xp1_1 = []
# models_resnet_xp1_1 = []
# for epoch in range(n_epochs):
#     train_loss, train_acc = train_step(model_resnet18,
#                                        optimizer_resnet18,
#                                        criterion_resnet18,
#                                        train_loader)
#     val_loss, val_acc = val_step(model_resnet18,
#                                   criterion_resnet18,
#                                   test_loader)
#     train_losses_resnet_xp1_1.append(train_loss)
#     val_losses_resnet_xp1_1.append(val_loss)
#     models_resnet_xp1_1 += [copy.deepcopy(model_resnet18)]

#     print_step_report(epoch=epoch,
#                       train_loss=train_loss,
#                       val_loss=val_loss,
#                       train_acc=train_acc,
#                       val_acc=val_acc)

# torch.save(models_resnet_xp1_1, "XP1/card_2-case_1/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_1 = []
val_losses_toy_xp1_1 = []
models_toy_xp1_1 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_1.append(train_loss)
    val_losses_toy_xp1_1.append(val_loss)
    models_toy_xp1_1 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_1, "XP1/card_2-case_1/toy_models.pth")

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
from devinterp.slt import estimate_learning_coeff

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_1:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

In [ ]:
plot_lambda_vs_epochs(train_losses_toy_xp1_1, val_losses_toy_xp1_1, rlct_estimates_sgnht, rlct_estimates_sgld)

##### 1.2 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{2, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_2)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_2 = []
val_losses_resnet_xp1_2 = []
models_resnet_xp1_2 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_2.append(train_loss)
    val_losses_resnet_xp1_2.append(val_loss)
    models_resnet_xp1_2 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_2, "XP1/card_2-case_2/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_2 = []
val_losses_toy_xp1_2 = []
models_toy_xp1_2 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_2.append(train_loss)
    val_losses_toy_xp1_2.append(val_loss)
    models_toy_xp1_2 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_2, "XP1/card_2-case_2/toy_models.pth")

##### 1.3 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{4, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_3)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_3 = []
val_losses_resnet_xp1_3 = []
models_resnet_xp1_3 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_3.append(train_loss)
    val_losses_resnet_xp1_3.append(val_loss)
    models_resnet_xp1_3 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_3, "XP1/card_2-case_3/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_3 = []
val_losses_toy_xp1_3 = []
models_toy_xp1_3 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_3.append(train_loss)
    val_losses_toy_xp1_3.append(val_loss)
    models_toy_xp1_3 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_3, "XP1/card_2-case_3/toy_models.pth")

##### 1.4 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{5, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_4)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_4 = []
val_losses_resnet_xp1_4 = []
models_resnet_xp1_4 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_4.append(train_loss)
    val_losses_resnet_xp1_4.append(val_loss)
    models_resnet_xp1_4 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_4, "XP1/card_2-case_4/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_4 = []
val_losses_toy_xp1_4 = []
models_toy_xp1_4 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_4.append(train_loss)
    val_losses_toy_xp1_4.append(val_loss)
    models_toy_xp1_4 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_4, "XP1/card_2-case_4/toy_models.pth")

##### 1.5 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_5)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_5 = []
val_losses_resnet_xp1_5 = []
models_resnet_xp1_5 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_5.append(train_loss)
    val_losses_resnet_xp1_5.append(val_loss)
    models_resnet_xp1_5 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_5, "XP1/card_2-case_5/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_5 = []
val_losses_toy_xp1_5 = []
models_toy_xp1_5 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_5.append(train_loss)
    val_losses_toy_xp1_5.append(val_loss)
    models_toy_xp1_5 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_5, "XP1/card_2-case_5/toy_models.pth")

### Settings and runs label sets $Card(\mathcal{L}abels)=3$

* Labels modification and data loader
* Toy model for 3 classes
* Resnet18 model for 3 classes

In [ ]:
# keys are the original label, values are the new labels
# [{2, 6, other} ; {3, 6, other} ; {0, 4, other} ; {8, 9, other} ; {5, 7, other}]
map_label_card_3_1 = {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 2, 7: 0, 8: 0, 9: 0}
map_label_card_3_2 = {0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 2, 7: 0, 8: 0, 9: 0}
map_label_card_3_3 = {0: 1, 1: 0, 2: 0, 3: 0, 4: 2, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_3_4 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 2}
map_label_card_3_5 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 2, 8: 0, 9: 0}

##### 2.1 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{2, 6, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_1)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_6 = []
val_losses_resnet_xp1_6 = []
models_resnet_xp1_6 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_6.append(train_loss)
    val_losses_resnet_xp1_6.append(val_loss)
    models_resnet_xp1_6 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_6, "XP1/card_3-case_1/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_6 = []
val_losses_toy_xp1_6 = []
models_toy_xp1_6 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_6.append(train_loss)
    val_losses_toy_xp1_6.append(val_loss)
    models_toy_xp1_6 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_6, "XP1/card_3-case_1/toy_models.pth")

##### 2.2 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{3, 6, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_7 = []
val_losses_resnet_xp1_7 = []
models_resnet_xp1_7 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_7.append(train_loss)
    val_losses_resnet_xp1_7.append(val_loss)
    models_resnet_xp1_7 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_7, "XP1/card_3-case_2/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_7 = []
val_losses_toy_xp1_7 = []
models_toy_xp1_7 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_7.append(train_loss)
    val_losses_toy_xp1_7.append(val_loss)
    models_toy_xp1_7 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_7, "XP1/card_3-case_2/toy_models.pth")

##### 2.3 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{0, 4, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_3)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_8 = []
val_losses_resnet_xp1_8 = []
models_resnet_xp1_8 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_8.append(train_loss)
    val_losses_resnet_xp1_8.append(val_loss)
    models_resnet_xp1_8 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_8, "XP1/card_3-case_3/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_8 = []
val_losses_toy_xp1_8 = []
models_toy_xp1_8 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_8.append(train_loss)
    val_losses_toy_xp1_8.append(val_loss)
    models_toy_xp1_8 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_8, "XP1/card_3-case_3/toy_models.pth")

##### 2.4 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{8, 9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_4)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_9 = []
val_losses_resnet_xp1_9 = []
models_resnet_xp1_9 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_9.append(train_loss)
    val_losses_resnet_xp1_9.append(val_loss)
    models_resnet_xp1_9 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_9, "XP1/card_3-case_4/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_9 = []
val_losses_toy_xp1_9 = []
models_toy_xp1_9 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_9.append(train_loss)
    val_losses_toy_xp1_9.append(val_loss)
    models_toy_xp1_9 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_9, "XP1/card_3-case_4/toy_models.pth")

##### 2.5 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{5, 7, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_5)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

model_toy, criterion_toy, optimizer_toy = models_factory(model_type="toy_model",
                                                         optim_cfg=dict_optim_cfg,
                                                         nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18

models_resnet, criterion_resnet, optimizer_resnet = models_factory(model_type="toy_model",
                                                                   optim_cfg=dict_optim_cfg,
                                                                   nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

train_losses_resnet_xp1_10 = []
val_losses_resnet_xp1_10 = []
models_resnet_xp1_10 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_resnet18,
                                       optimizer_resnet18,
                                       criterion_resnet18,
                                       train_loader)
    val_loss, val_acc = val_step(model_resnet18,
                                  criterion_resnet18,
                                  test_loader)
    train_losses_resnet_xp1_10.append(train_loss)
    val_losses_resnet_xp1_10.append(val_loss)
    models_resnet_xp1_10 += [copy.deepcopy(model_resnet18)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_resnet_xp1_10, "XP1/card_3-case_5/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
train_losses_toy_xp1_10 = []
val_losses_toy_xp1_10 = []
models_toy_xp1_10 = []
for epoch in range(n_epochs):
    train_loss, train_acc = train_step(model_toy,
                                       optimizer_toy,
                                       criterion_toy,
                                       train_loader)
    val_loss, val_acc = val_step(model_toy,
                                  criterion_toy,
                                  test_loader)
    train_losses_toy_xp1_10.append(train_loss)
    val_losses_toy_xp1_10.append(val_loss)
    models_toy_xp1_10 += [copy.deepcopy(model_toy)]

    print_step_report(epoch=epoch,
                      train_loss=train_loss,
                      val_loss=val_loss,
                      train_acc=train_acc,
                      val_acc=val_acc)

torch.save(models_toy_xp1_10, "XP1/card_3-case_5/toy_models.pth")

### Settings and runs label sets $Card(\mathcal{L}abels)=4$

* Labels modification and data loader
* Toy model for 4 classes
* Resnet18 model for 4 classes

In [ ]:
# keys are the original label, values are the new labels
# [{2, 6, other} ; {3, 6, other} ; {0, 4, other} ; {8, 9, other} ; {5, 7, other}]
map_label_card_4_1 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_4_2 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_4_3 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_4_4 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_4_5 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}

### Settings and runs label sets $Card(\mathcal{L}abels)=5$

* Labels modification and data loader
* Toy model for 5 classes
* Resnet18 model for 5 classes

### Settings and runs label sets $Card(\mathcal{L}abels)=10$

* Labels modification and data loader
* Toy model for 10 classes
* Resnet18 model for 10 classes

## Displays λ 𝑣𝑠 $\mathcal{L}$𝑎𝑏𝑒𝑙𝑠

In [ ]:
sns.set_style("whitegrid")

fig, ax1 = plt.subplots()
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss", color=PRIMARY)
ax1.plot(train_losses, label="Train Loss", color=PRIMARY)
ax1.plot(test_losses, label="Test Loss", color=PRIMARY_LIGHT)
ax1.tick_params(axis="y", labelcolor=PRIMARY)
ax1.legend(loc="lower left")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Local Learning Coefficient, $\hat \lambda$", color=SECONDARY)
ax2.plot(rlct_estimates_sgnht, label="SGNHT", color=SECONDARY)
ax2.plot(rlct_estimates_sgld, label="SGLD", color=SECONDARY_LIGHT)
ax2.tick_params(axis="y", labelcolor=SECONDARY)
ax2.legend(loc="center right")

fig.tight_layout()
plt.show()